<a href="https://colab.research.google.com/github/GuAparecido/Projeto_Final_Python/blob/main/ProjetoFinalPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import hashlib